In [1]:
contest_folder = 'practice'

In [2]:
from analysis import get_current_status, select_solution

In [3]:
df_grouped, aggregated_df = get_current_status(contest_folder)

In [4]:
aggregated_list = aggregated_df.to_dict(orient='records')

In [5]:
aggregated_df

,problem_code,problem_name,statement,sample_in,sample_out,full_in,solution_id,execution_response,execution_code,execution_sample_out,execution_full_out,hash,status,timestring
0,fall_in_line,Fall in Line,"As the queen of an ant colony, it’s your job t...",3\n4\n1 1\n2 2\n-3 -3\n4 4\n4\n1 1\n-1 1\n1 -1...,Case #1: 0\nCase #2: 2\nCase #3: 3\n,113\n4\n1 1\n2 2\n-3 -3\n4 4\n4\n1 1\n-1 1\n1 ...,"[000, 001, 002, 003, 004]",[**Key Findings:**\n\nTo solve this problem ef...,[import sys\nimport math\nimport random\nfrom ...,"[Case #1: 0\nCase #2: 2\nCase #3: 3\n, Case #1...",[Case #1: 0\nCase #2: 2\nCase #3: 3\nCase #4: ...,d3fbbc49,not_analyzed,5241
1,line_by_line,Line by Line,You’ve found a solution to an implementation-h...,4\n2 50\n3 10\n13 37\n950 95\n,Case #1: 20.710678118654748\nCase #2: 11.54434...,174\n2 50\n3 10\n13 37\n950 95\n2 1\n2 2\n2 3\...,"[000, 001, 002, 003, 004]",[**Key Findings:**\n\n1. **Probability Calcula...,[import math\nimport sys\n\ndef main():\n T...,[Case #1: 20.710678118654755\nCase #2: 11.5443...,[Case #1: 20.710678118654755\nCase #2: 11.5443...,2e817376,not_analyzed,5241
2,line_of_delivery_part_1,Line of Delivery (Part 1),*This problem shares some similarities with pr...,4\n2 5\n7\n2\n3 1\n9\n5\n7\n4 7\n8\n7\n9\n6\n2...,Case #1: 1 2\nCase #2: 3 4\nCase #3: 3 0\nCase...,111\n2 5\n7\n2\n3 1\n9\n5\n7\n4 7\n8\n7\n9\n6\...,"[000, 001, 002]","[To solve this problem efficiently, we need to...",[import sys\nimport threading\n\ndef main():\n...,[Case #1: 1 2\nCase #2: 2 4\nCase #3: 2 0\nCas...,[Case #1: 1 2\nCase #2: 2 4\nCase #3: 2 0\nCas...,f4132b60,not_analyzed,5241
3,line_of_delivery_part_2,Line of Delivery (Part 2),*This problem shares some similarities with pr...,5\n2 5\n7\n2\n3 6\n9\n5\n7\n4 8\n8\n7\n9\n6\n2...,Case #1: 1 2\nCase #2: 2 2\nCase #3: 2 0\nCase...,112\n2 5\n7\n2\n3 6\n9\n5\n7\n4 8\n8\n7\n9\n6\...,"[000, 001, 002]",[**Key Findings:**\n\n1. **Efficient Simulatio...,[import sys\nimport sys\nimport sys\nimport sy...,[Case #1: 2 4\nCase #2: 3 4\nCase #3: 4 5\nCas...,[Case #1: 2 4\nCase #2: 3 4\nCase #3: 4 5\nCas...,5ce0a7e7,not_analyzed,5241
4,walk_the_line,Walk the Line,"There’s an old, narrow bridge that a group of ...",6\n4 17\n1\n2\n5\n10\n4 4\n1\n2\n5\n10\n2 22\n...,Case #1: YES\nCase #2: NO\nCase #3: YES\nCase ...,114\n4 17\n1\n2\n5\n10\n4 4\n1\n2\n5\n10\n2 22...,"[001, 002, 004]","[To solve this problem, we need to calculate t...","[T = int(input())\nfor case_num in range(1, T ...",[Case #1: YES\nCase #2: NO\nCase #3: YES\nCase...,[Case #1: YES\nCase #2: NO\nCase #3: YES\nCase...,80615b62,not_analyzed,5241


In [6]:
import time
import threading

while True:
    _, aggregated_df = get_current_status(contest_folder)
    if len(aggregated_df) == 0:
        print("Not outputs found yet")
        time.sleep(30)
        continue
    
    aggregated_list = aggregated_df.to_dict(orient='records')
    print(f"Running analysis at {aggregated_df['timestring'][0]}")
    
    for row in aggregated_list:
        if row["status"] != "not_analyzed":
            continue
        analysis_thread = threading.Thread(target=select_solution, args=(row,))
        analysis_thread.start()
        print(f"Running analysis at {row['timestring']} for {row['problem_name']}")

    time.sleep(30)

Running analysis at 5242
Running analysis at 5242 for Fall in Line
Running analysis at 5242 for Line by Line
Running analysis at 5242 for Line of Delivery (Part 1)
Running analysis at 5242 for Line of Delivery (Part 2)
Running analysis at 5242 for Walk the Line
Selected solution 001 for walk_the_line
Selected solution 000 for line_of_delivery_part_1
Selected solution 000 for line_of_delivery_part_2
Selected solution 001 for fall_in_line
Selected solution 003 for line_by_line
Running analysis at 5313
Running analysis at 5344
Running analysis at 5415
Running analysis at 5445
Running analysis at 5516


KeyboardInterrupt: 